In [1]:
# Place %autoreload 2 at the top of your notebook to ensure it applies to all subsequent imports.
%load_ext autoreload
%autoreload 2
 


In [2]:
from __future__ import absolute_import, division, print_function
import torch
from trainer_end_to_end import Trainer
from options import MonodepthOptions
import time
options = MonodepthOptions()
args = [
    '--data_path', '/mnt/nct-zfs/TCO-All/SharedDatasets/SCARED_Images_Resized/',
    # '--of_samples_num', '3', 
    '--batch_size', '10', 
    # '--log_frequency', '2',
    # '--log_dir', '/mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/debug_reloc3r_backbone/relocxr',
    '--debug',
    ]
opts = options.parse_notebook(args)# trainer.set_eval()
# opts

2025-08-29 10:17:24.210722: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-29 10:17:24.222858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756455444.237575   70465 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756455444.241812   70465 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756455444.252932   70465 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


In [3]:
# load model
trainer = Trainer(opts)
print('trainer initialized...')

DEBUG MODE
update options for debug purposes...
1234 is the seed!
learning rate: 0.0001
batch size is: 1
Using DoMoRA
DARES_MoDoRA


/mnt/cluster/environments/jinjingxu/pkg/envs/cu12/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


DoRA params initialized!
DoRA rank is: [14, 14, 12, 12, 10, 10, 8, 8, 8, 8, 8, 8]
DAM mora rank is  [14, 14, 12, 12, 10, 10, 8, 8, 8, 8, 8, 8]
Vector MoRA params initialized!
MoDoRA in Depth model


/mnt/cluster/workspaces/jinjingxu/proj/UniSfMLearner/submodule/Endo_FASt3r/trainer_end_to_end.py:174: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.models["position_enc

DoMoRA in Reloc3rX


/mnt/cluster/workspaces/jinjingxu/proj/UniSfMLearner/submodule/Endo_FASt3r/reloc3r/reloc3r_relpose.py:182: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ck

Model loaded from  /mnt/cluster/workspaces/jinjingxu/proj/MVP3R/baselines/reloc3r/checkpoints/reloc3r-512/Reloc3r-512.pth
cross attention layers are frozen!
DoRA params initialized!
encoder rank is [20, 20, 20, 18, 18, 18, 18, 18, 16, 16, 16, 16, 14, 14, 12, 12, 10, 10, 8, 8, 8, 8, 8, 8]
decoder rank is [14, 14, 12, 12, 10, 10, 8, 8, 8, 8, 8, 8]
DAM mora rank for enc is  [20, 20, 20, 18, 18, 18, 18, 18, 16, 16, 16, 16, 14, 14, 12, 12, 10, 10, 8, 8, 8, 8, 8, 8]
DAM mora rank for dec is  [14, 14, 12, 12, 10, 10, 8, 8, 8, 8, 8, 8]
Vector MoRA params initialized!
CHECK: NO self.opt.predictive_mask
Training model named:
   debug
Models and tensorboard events files are saved to:
   /mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/debug_reloc3r_backbone/relocxr
Training is using:
   cuda
use only of_samples_num samples for OF, for OF training...
using 10 samples for OF, for speed up training..
use only of_samples_num samples for OF, for OF training...
using 10 samples for OF, for spee

/mnt/cluster/environments/jinjingxu/pkg/envs/cu12/lib/python3.11/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647348947/work/aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
# opts.pose_model_type
trainer.opt.pose_model_type

In [11]:
# trainer = Trainer(opts)
self = trainer #Trainer(opts)
print('trainer initialized...')









# %%

# trainer.train()
"""Run the entire training pipeline
"""
self.epoch = 0
self.step = 0
self.start_time = time.time()
with torch.no_grad():
    # for self.epoch in range(self.opt.num_epochs):
    # self.run_epoch(self.epoch)
    """Run a single epoch of training and validation
    """
    epoch = self.epoch
    print("Training")

    self.model_optimizer_0.zero_grad()# debug only: save_mem
    self.model_optimizer.zero_grad()# debug only: save_mem

    # for batch_idx, inputs in enumerate(self.train_loader):

    train_iter = iter(self.train_loader)
    inputs = next(train_iter)
    before_op_time = time.time()

    # position
    self.set_train_0()
    if self.opt.debug:
        self.freeze_params(keys = ['position_encoder'])#debug only to save_mem
    _, losses_0 = self.process_batch_0(inputs)
    self.model_optimizer_0.zero_grad()
    # losses_0["loss"].backward()#debug only
    torch.nn.utils.clip_grad_norm_(self.parameters_to_train_0, max_norm=1.0)
    self.model_optimizer_0.step()

    self.set_train()
    if self.opt.debug:
        self.freeze_params(keys = ['depth_model', 'pose', 'transform_encoder'])#debug only to save_mem
    outputs, losses = self.process_batch(inputs)
    self.model_optimizer.zero_grad()
    # losses["loss"].backward() #debug only
    torch.nn.utils.clip_grad_norm_(self.parameters_to_train, max_norm=1.0)
    self.model_optimizer.step()

    duration = time.time() - before_op_time

    # phase = batch_idx % self.opt.log_frequency == 0

    # if phase:

    # self.log_time(batch_idx, duration, losses["loss"].cpu().data)
    concat_img, img_order_strs = self.log("train", inputs, outputs, losses, compute_vis=True, online_vis=False)

    import os, cv2
    save_path = "/mnt/cluster/workspaces/jinjingxu/proj/UniSfMLearner/submodule/Endo_FASt3r/"
    save_path = os.path.join(save_path, f"{img_order_strs}.png")
    cv2.imwrite(save_path, concat_img)
    print(f"saved train_concat_img.png in {save_path}")

    self.step += 1
    
    # break
        
    self.model_lr_scheduler.step()
    self.model_lr_scheduler_0.step()

    # if (self.epoch + 1) % self.opt.save_frequency == 0:
    #     self.save_model()
    # break


trainer initialized...
Training
color shape: torch.Size([3, 192, 224])
motion_flow shape: torch.Size([2, 192, 224])
position shape: torch.Size([2, 192, 224])
color shape: torch.Size([3, 192, 224])
motion_flow shape: torch.Size([2, 192, 224])
position shape: torch.Size([2, 192, 224])
Optic flow:
Pose flow:
Motion flow:
saved train_concat_img.png in /mnt/cluster/workspaces/jinjingxu/proj/UniSfMLearner/submodule/Endo_FASt3r/Src-Tgt-Registered_Tgt-Colored_Tgt-Colored_Motion_Tgt-Optic_Flow-Pose_Flow-Motion_Flow-Occlursion_Mask.png


In [6]:

# self.set_eval()
# try:
#     # inputs = self.val_iter.next()
#     inputs = next(self.val_iter)
# except StopIteration:
#     self.val_iter = iter(self.val_loader)
#     inputs = self.val_iter.next()

# with torch.no_grad():
#     # outputs, losses = self.process_batch_val(inputs)
#     """Pass a minibatch through the network and generate images and losses
#     """
#     for key, ipt in inputs.items():
#         inputs[key] = ipt.to(self.device)

#     if self.opt.pose_model_type == "shared":
#         # If we are using a shared encoder for both depth and pose (as advocated
#         # in monodepthv1), then all images are fed separately through the depth encoder.
#         print('CHECK: self.opt.pose_model_type == "shared"')
#         all_color_aug = torch.cat([inputs[("color_aug", i, 0)] for i in self.opt.frame_ids])
#         all_features = self.models["encoder"](all_color_aug)
#         all_features = [torch.split(f, self.opt.batch_size) for f in all_features]

#         features = {}
#         for i, k in enumerate(self.opt.frame_ids):
#             features[k] = [f[i] for f in all_features]

#         outputs = self.models["depth"](features[0])
#     else:
#         print('CHECK: self.opt.pose_model_type != "shared"')
#         # Otherwise, we only feed the image with frame_id 0 through the depth encoder
#         features = self.models["encoder"](inputs["color_aug", 0, 0])
#         outputs = self.models["depth"](features)

#     if self.opt.predictive_mask:
#         outputs["predictive_mask"] = self.models["predictive_mask"](features)

#     if self.use_pose_net:
#         outputs.update(self.predict_poses(inputs, features, outputs))

#     self.generate_images_pred(inputs, outputs)
#     losses = self.compute_losses_val(inputs, outputs)

#     # return outputs, losses

#     self.log("val", inputs, outputs, losses)
#     del inputs, outputs, losses